In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras import Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.saving import load_model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

## Exploratory Visualization and Analysis

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
single_image = x_train[0]

In [ ]:
single_image.shape

In [ ]:
plt.imshow(single_image);

## Labeling

In [ ]:
y_train

In [ ]:
y_train.shape

In [ ]:
y_cat_train = to_categorical(y_train, 10)
y_cat_test = to_categorical(y_test, 10)

In [ ]:
y_cat_train

In [ ]:
y_cat_train.shape

## Scaling and Reshaping

In [ ]:
x_train.max()

In [ ]:
x_train.min()

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
plt.imshow(x_train[0]);

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

## Creating a Custom Model and Training

In [ ]:
model = Sequential()

model.add(Input(shape=(28,28,1)))

model.add(Conv2D(filters=32, kernel_size=(4,4), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(4,4), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(4,4), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax'))

    Categorical Cross Entropy:
    Use this crossentropy loss function when there are two or more label classes. We expect labels to be provided in a one_hot representation.

    y_true = [[0, 1, 0], [0, 0, 1]]
    y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]

    Sparse Categorical Cross Entropy:
    Use this crossentropy loss function when there are two or more label classes. We expect labels to be provided as integers.

    y_true = [1, 2]
    y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.fit(x=x_train, y=y_cat_train, epochs=15, batch_size=32, validation_data=(x_test,y_cat_test), callbacks=[early_stop])

## Save Model

In [ ]:
model.save('C:\\Users\\alvar\\Documents\\tf_templates\\CNN\\mnist_model.keras')

In [ ]:
my_model = load_model('C:\\Users\\alvar\\Documents\\tf_templates\\CNN\\mnist_model.keras')

## Evaluation

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['accuracy','val_accuracy']].plot();

In [ ]:
losses[['loss','val_loss']].plot();

In [ ]:
print(model.metrics_names)
print(model.evaluate(x_test, y_cat_test, verbose=0))

In [ ]:
predictions = np.argmax(model.predict(x_test), axis=1)
predictions

In [ ]:
y_test

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(confusion_matrix(y_test, predictions), annot=True);

## Predict Classes

In [ ]:
my_number = x_test[100]
plt.imshow(my_number.reshape(28,28,1));

In [ ]:
prediction = np.argmax(my_model.predict(my_number.reshape(1,28,28,1)), axis=1)
prediction